In [3]:
import numpy as np
import math
import random 
from random import randint
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from IPython.display import HTML 
import scipy
from scipy import special
from scipy.integrate import odeint 
import scipy.integrate as integrate
from scipy.integrate import dblquad
import statistics
from sympy import sin, cos, pi
from tqdm import tqdm

import subprocess
import os

#%matplotlib inline
#%matplotlib widget

**TREE-CODE**

This algorithm does an approximation on the force computation. 
Given N particles in a volume V, tree-code devides teh volume iteratively into cubes up to the point each one contains only 1 particle. 
When the force acting over one particle i is computed, particles sufficiently far away from it are grouped in one point (the center ??) and their mass is sum up in total mass M. This is equal to take only the first order of the multipolar expansion (monopole term).

Computational cost: $N\log(N)$

Leaf opening criterion: $\theta = r_{cell}/d < \theta_{crit}$ 

$\theta_{crit}$ is the accuracy parameter (<<1 radiant)

Gravitational softening: to avoid huge scatterings we set the force different from newtonian one if the particles get to much close:

$|f_{ij}|=\frac{Gm_im_j}{|r_i-r_j|^2+\epsilon^2}$

does not diverge: it's like the mass of each particle has been smoothed over a finite volume.

Time integration: LEAP-FROG method. $t_i -> t_{i+1}$

$v_{i+\frac{1}{2}}=v_i+(\frac{h}{2})a_i$

$r_{i+1}=r_i+hv_{i+\frac{1}{2}}$

$v_{i+1}=v_{i+\frac{1}{2}}+(\frac{h}{2})a_i$

h is NOT ADAPTIVE, it's set at imput. We will set $h<<t_{dyn}$

$\epsilon = 10^{-4}(V/N)^{1/3}$

**INTERNAL UNITS**

nbody_sh1 and treecode work with internal units. $G_{iu}$ is set equal to 1. We are free to express masses and distances in arbitrary units, as far as velocities and times are consistent. In order to see that, notice that $\frac{GM}{rv^2}$ is adimensional. 

$$\frac{M_{iu}}{r_{iu}v_{iu}^2}=\frac{G_{cgs}M_{cgs}}{r_{cgs}v_{cgs}^2}$$

Velocities in internal units will be related to physical velocities by:

$$v_{iu}=\sqrt{\frac{r_{cgs}}{G_{cgs}M_{cgs}}}v_{cgs}$$

We need to tranform time units in internal units as well. $\frac{rt}{v}$ is adimensional. 

$$\frac{t_{iu}v_{iu}}{r_{iu}}=\frac{t_{cgs}v_{cgs}}{r_{cgs}}$$

$$t_{iu}=\frac{r_{iu}}{r_{cgs}}\sqrt{\frac{r_{cgs}}{G_{cgs}M_{cgs}}}t_{cgs}$$

In [4]:
c_cgs = 2.99792458 * 10**10 #cm/s
G_cgs = 6.67259 * 10**-8 #G in cgs
M_sun = 1.9891 * 10**33 #solar mass in g
R_sun = 6.9598 * 10**10 #solar radius in cm 
M_earth = 5.976 * 10**27 #earth mass in g
R_earth = 6.378 * 10**8 #earth radius in cm
ly = 9.463 * 10**17 #light year in cm
parsec = 3.086 * 10**18 #parsec in cm
AU = 1.496 * 10**13 #astronomical unit in cm
Sun_shw = 2*G_cgs*M_sun/(c_cgs**2) #Sun's Schwarschild radius in cgs

kB = 1.380649 * 10**-16 #erg/K

def v_IU(M_cgs, r_cgs, v_cgs):
    return np.sqrt(r_cgs/(G_cgs*M_cgs))*v_cgs

def c_IU(M_cgs, r_cgs):
    return np.sqrt(r_cgs/(G_cgs*M_cgs))*c_cgs

def t_IU(M_cgs, r_cgs, t_cgs):
    return t_cgs/(np.sqrt(r_cgs/(G_cgs*M_cgs))*r_cgs)

#print("If we choose 1 solar mass as mass unit and 1 parsec unit as distance unit:")
#print("1 Myr expressed in internal units is: %f" % (t_IU(M_sun,parsec,3.156*10**13)))
#print("1 time unit expressed in cgs is: %f Myrs" % (1/t_IU(M_sun,parsec,3.156*10**13)))
#print("1 time unit expressed in cgs is: %f s" % (1/t_IU(M_sun,parsec,1)))
#print("Light speed expressed in internal units is: %E" % (c_IU(M_sun,parsec)))
#print("Sun rest mass energy in internal units is: %E" %(1*c_IU(M_sun,parsec)))
#print("Sun kinetic energy (0.5M(200km/s) = %E erg) in internal units is: %E" %(0.5*M_sun*(200*10**5)**2, 0.5*v_IU(M_sun, parsec, 200*10**5)))
#print("1 km/s in internal units is: %f" %(v_IU(M_sun, parsec, 1*10**5)))


In [ ]:
#print("If we choose 1 solar mass as mass unit and Sun's schwarzschild radius as distance unit:")
#print("1 s expressed in internal units is: %f" % (t_IU(M_sun,Sun_shw,1)))
#print("1 time unit expressed in cgs is: %f s" % (1/t_IU(M_sun,Sun_shw,1)))
#print("Light speed expressed in internal units is: %E" % (c_IU(M_sun,Sun_shw)))
#print("Sun rest mass energy in internal units is: %E" %(1*c_IU(M_sun,Sun_shw)))
#print("Sun kinetic energy (0.5M(200km/s) = %E erg) in internal units is: %E" %(0.5*M_sun*(200*10**5)**2, 0.5*v_IU(M_sun, Sun_shw, 200*10**5)))

In [5]:
time_conv = 0.067069 #1 Myr in internal units
energy_conv = 1/1524.834 #sun kinetic energy in internal units
velocity_conv = 15.248335 #Km/s in internal units

In [ ]:
class Star:
    def __init__(self, m, x, v): #x is the cartesian coordinate vector, v the cartesian velocity
        self.m = m
        self.position = x
        self.velocity = v
        self.r = np.sqrt(x[0]**2+x[1]**2+x[2]**2) 
        self.v = np.sqrt(v[0]**2+v[1]**2+v[2]**2) #magnitude of velocity
        self.kinetic_energy = (0.5)*self.m*self.v**2 #expressed in internal units
        self.L = self.m*np.array([x[1]*v[2]-x[2]*v[1],x[2]*v[0]-x[0]*v[2],x[0]*v[1]-x[1]*v[0]])
        self.l = np.sqrt(self.L[0]**2+self.L[1]**2+self.L[2]**2)



class Galaxy:
    def __init__(self,N):
        self.N = N
        self.n_body_system = [] #list of stars objects
        self.P = [] #list of position objects
        self.t = [0] 
        self.E_tot = [] ; self.T = [] ; self.U = [] ; self.T_U_ratio = []
        self.dynamical_time = 0 ; self.t_cross = 0 ; self.t_relax = 0 #will be initialized
        self.perturber_index = []

In [ ]:
def perturber(galaxy,M_p,R_p,V_p):
    pert = Star(M_p,R_p,V_p)
    galaxy.n_body_system[0] = np.append(galaxy.n_body_system[0], pert)
    galaxy.perturber_index.append(galaxy.N) #index at which the perturber is
    galaxy.N = galaxy.N + 1
    galaxy.perturber = True


In [ ]:
def merger(galaxy_1,galaxy_2,r1,r2,v1,v2):

    N = galaxy_1.N + galaxy_2.N
    eps = min(galaxy_1.eps,galaxy_2.eps)
    v_lim = 50 #????

    galactic_merger = Galaxy(N,eps,v_lim,"merger")
    
    initial_system = []
    for star in galaxy_1.n_body_system[0]:
        star.position = star.position + r1
        star.velocity = star.velocity + v1
        initial_system.append(star)
    
    for star in galaxy_2.n_body_system[0]:
        star.position = star.position + r2
        star.velocity = star.velocity + v2
        initial_system.append(star)

    galactic_merger.n_body_system.append(initial_system)
    for i in range(len(galaxy_2.perturber_index)):
        galaxy_2.perturber_index[i] = galaxy_2.perturber_index[i] + galaxy_1.N
    galactic_merger.perturber_index = galaxy_1.perturber_index + galaxy_2.perturber_index

    return galactic_merger

In [ ]:
def evolve_treecode(galaxy,filename,tstart,tstop,dtime,eps,theta=1):
    dtout=dtime
        
    if os.path.exists("%s.in" %filename):
      os.remove("%s.in" %filename) 
    if os.path.exists("%s.out" %filename):
      os.remove("%s.out" %filename)

    if tstart > galaxy.t[-1]: tstart = galaxy.t[-1] 
    initial_frame = np.digitize(tstart,galaxy.t)-1
    del(galaxy.n_body_system[initial_frame+1:])
    del(galaxy.t[initial_frame+1:])
    
    #writes initial conditions in file for treecode
    initial_conditions_file = open("%s.in" %filename, "w") 
    initial_conditions_file.write(str(galaxy.N)+"\n") #number of bodies
    initial_conditions_file.write(str(3)+"\n") #dimensions 
    initial_conditions_file.write(str(tstart)+"\n") #initial time
    
    for star in galaxy.n_body_system[initial_frame]:
        initial_conditions_file.write(str(star.m)+"\n")   
    for star in galaxy.n_body_system[initial_frame]:
        initial_conditions_file.write(str(star.position[0])+"\t"+str(star.position[1])+"\t"+str(star.position[2])+"\n")
    for star in galaxy.n_body_system[initial_frame]:
        initial_conditions_file.write(str(star.velocity[0])+"\t"+str(star.velocity[1])+"\t"+str(star.velocity[2])+"\n")
    initial_conditions_file.close()

    logfile = open("%s_logfile" %filename, "w")
    print("./treecode in=%s.in out=%s.out dtime=%f theta=%f eps=%f tstop=%f dtout=%f >%s.logfile" %(filename,filename,dtime,theta,eps,tstop,dtout,filename))
    subprocess.call(["./treecode", "in=%s.in" %filename, "out=%s.out" %filename, "dtime=%f" %dtime, "theta=%f" %theta, "eps=%f" %eps, "tstop=%f" %tstop, "dtout=%f" %dtout],stdout=logfile) 

    
    
    #cancels initial conditions and writes them again
    
    del(galaxy.n_body_system[-1])
    del(galaxy.t[-1])
    
    #opens the output file and writes star's positions array for each time in t
    
    output = open("%s.out" %filename, "r").readlines()
    
    #makes array of array of star
    for i in range(int(len(output)/(3*galaxy.N+3))):
        galaxy.t.append(float(output[i*(3*galaxy.N+3)+2])) #time in internal units
        stars = []
        for j in range(galaxy.N): 
            m = output[i*(3*galaxy.N+3)+3+j]
            p = output[i*(3*galaxy.N+3)+3+galaxy.N+j]
            #float(output[i*(self.N+2)+(2+j)].split()[1])
            v = output[i*(3*galaxy.N+3)+3+2*galaxy.N+j]
            stars.append(Star(float(m),[float(p.split()[0]),float(p.split()[1]),float(p.split()[2])],[float(v.split()[0]),float(v.split()[1]),float(v.split()[2])]))     
        galaxy.n_body_system.append(stars)
            
    #opens logfile and writes energy for each time in t 
    #output = open("%s_logfile" %filename, "r").readlines()
    #for i in range(len(galaxy.n_body_system)): #-2??
        #galaxy.t.append(float(output[10+i*8].split()[0])) #time in internal units
        #galaxy.E_tot.append(float(output[10+i*8].split()[1]))
        #galaxy.T.append(float(output[10+i*8].split()[2]))
        #galaxy.U.append(float(output[10+i*8].split()[3]))
        #galaxy.T_U_ratio.append(float(output[10+i*8].split()[4]))
    #self.t = np.array(self.t) 

    #creates array very usefull for animations
    positions_at_given_frame = []
    for frame in galaxy.n_body_system:
        position_of_nth_star = []
        for star in frame:
            position_of_nth_star.append([star.position[0],star.position[1],star.position[2]])
        positions_at_given_frame.append(np.array(position_of_nth_star))
    galaxy.P = np.array(positions_at_given_frame)

    print("Galaxy evolved to time %.2f: %s frames" %(galaxy.t[-1],len(galaxy.n_body_system)))

In [ ]:
def init_treecode(galaxy,filename):
    if os.path.exists("%s.in" %filename):
      os.remove("%s.in" %filename) 
    if os.path.exists("%s.out" %filename):
      os.remove("%s.out" %filename)

    #if tstart > galaxy.t[-1]: tstart = galaxy.t[-1] 
    #initial_frame = np.digitize(tstart,galaxy.t)-1
    #del(galaxy.n_body_system[initial_frame+1:])
    #del(galaxy.t[initial_frame+1:])
    
    #writes initial conditions in file for treecode
    initial_conditions_file = open("%s.in" %filename, "w") 
    initial_conditions_file.write(str(galaxy.N)+"\n") #number of bodies
    initial_conditions_file.write(str(3)+"\n") #dimensions 
    initial_conditions_file.write(str(0)+"\n") #initial time
    
    for star in galaxy.n_body_system[0]:
        initial_conditions_file.write(str(star.m)+"\n")   
    for star in galaxy.n_body_system[0]:
        initial_conditions_file.write(str(star.position[0])+"\t"+str(star.position[1])+"\t"+str(star.position[2])+"\n")
    for star in galaxy.n_body_system[0]:
        initial_conditions_file.write(str(star.velocity[0])+"\t"+str(star.velocity[1])+"\t"+str(star.velocity[2])+"\n")
    initial_conditions_file.close()

In [ ]:

### for uploading long calculations

def upload_treecode(galaxy,filename,step=1): #if output already existent

    #cancels initial conditions and writes them again
    if len(galaxy.n_body_system)>0:
        del(galaxy.n_body_system[-1])
        del(galaxy.t[-1])
        
    #opens the output file and writes star's positions array for each time in t
    output = open("%s.out" %filename, "r").readlines()
    #makes array of array of star
    for i in range(int(len(output)/(3*galaxy.N+3)/step)-step):
        i_ = i*step
        galaxy.t.append(float(output[i_*step*(3*galaxy.N+3)+2])) #time in internal units
        stars = []
        for j in range(galaxy.N): 
            m = output[i_*(3*galaxy.N+3)+3+j]
            p = output[i_*(3*galaxy.N+3)+3+galaxy.N+j]
            #float(output[i*(self.N+2)+(2+j)].split()[1])
            v = output[i_*(3*galaxy.N+3)+3+2*galaxy.N+j]
            stars.append(Star(float(m),[float(p.split()[0]),float(p.split()[1]),float(p.split()[2])],[float(v.split()[0]),float(v.split()[1]),float(v.split()[2])]))
        galaxy.n_body_system.append(stars)

    #creates array very usefull for animations
    positions_at_given_frame = []
    for frame in galaxy.n_body_system:
        position_of_nth_star = []
        for star in frame:
            position_of_nth_star.append([star.position[0],star.position[1],star.position[2]])
        positions_at_given_frame.append(np.array(position_of_nth_star))
    galaxy.P = np.array(positions_at_given_frame)
    
    print("Galaxy evolved to time %.2f: %s frames" %(galaxy.t[-1],len(galaxy.n_body_system)))


In [ ]:

def lagrangian_radius(galaxy,percentage,time=0):
    
    if time > galaxy.t[-1]: time=galaxy.t[-1] 
    frame = np.digitize(time,galaxy.t)-1
    
    R = []
    for i in range(len(galaxy.n_body_system[frame])):
        R.append(np.sqrt(np.dot(galaxy.n_body_system[frame][i].position-galaxy.n_body_system[frame][0].position,galaxy.n_body_system[frame][i].position-galaxy.n_body_system[frame][0].position)))
    
    return np.percentile(R,percentage)
        
    

In [ ]:

def virial(galaxy,time=0):
    
    if time > galaxy.t[-1]: time=galaxy.t[-1] 
    frame = np.digitize(time,galaxy.t)-1
    
    Kinetic_energy = []
    Potential_energy = []
    for star in galaxy.n_body_system[frame]:
        Kinetic_energy.append(star.kinetic_energy)
        potential_sum = 0
        for star_ in galaxy.n_body_system[frame]:
            d = np.sqrt(np.dot(np.array(star.position)-np.array(star_.position),np.array(star.position)-np.array(star_.position)))
            if d != 0:
                potential_sum += star.m*star_.m/d
        Potential_energy.append(potential_sum)

    K = np.sum(Kinetic_energy) 
    U = - 1/2 * np.sum(Potential_energy)  #cicle counted each couple twice
    print(K/U)



In [ ]:
def view(galaxy, r_max, time=0, orbits_radii = [], r_orbit_min=0., r_orbit_max=0.):

    if time > galaxy.t[-1]: time=galaxy.t[-1] 
    frame = np.digitize(time,galaxy.t)-1

    index_orbits = []
    for r in orbits_radii: 
        index = 0
        r_min = r_max
        for i in range(len(galaxy.n_body_system[0])):
            if np.absolute(r-galaxy.n_body_system[0][i].r)<r_min:
                r_min = np.absolute(r-galaxy.n_body_system[0][i].r)
                index = i
        index_orbits.append(index)
                    
    fig = plt.figure(figsize=(10, 10)) ; ax = fig.add_subplot(projection='3d') ; 
    ax.set_xlim3d(-r_max, r_max) ; ax.set_ylim3d(-r_max, r_max) ; ax.set_zlim3d(-r_max, r_max) 
    ax.set_xlabel('X [parsec]') ; ax.set_ylabel('Y [parsec]') ; ax.set_zlabel('Z [parsec]') ; ax.set_aspect("equal") ;  # Provide starting angle for the view. #ax.view_init(25, 10)
    plt.title("t = %.2f i.u. (%.2f Myr)" %(time,time*time_conv), fontsize=10) 
    
    ax.scatter(galaxy.P[frame,:,0],galaxy.P[frame,:,1],galaxy.P[frame,:,2],c='black',s=0.5)
    for j in index_orbits:
        plt.plot(galaxy.P[:frame,j,0],galaxy.P[:frame,j,1],galaxy.P[:frame,j,2])
        ax.scatter(galaxy.P[frame,j,0],galaxy.P[frame,j,1],galaxy.P[frame,j,2],s=5)

    for k in galaxy.perturber_index:
        plt.plot(galaxy.P[:frame,k,0],galaxy.P[:frame,k,1],galaxy.P[:frame,k,2],c='red')
        ax.scatter(galaxy.P[frame,k,0],galaxy.P[frame,k,1],galaxy.P[frame,k,2],c='red',s=50)

    if r_orbit_max > 0:
        orbits_enclosured = []
        for i in range(len(galaxy.n_body_system[0])):
            if galaxy.n_body_system[0][i].r > r_orbit_min and galaxy.n_body_system[0][i].r < r_orbit_max:
                orbits_enclosured.append(i)
        for i in orbits_enclosured:
            plt.plot(galaxy.P[:frame,i,0],galaxy.P[:frame,i,1],galaxy.P[:frame,i,2])
            ax.scatter(galaxy.P[frame,i,0],galaxy.P[frame,i,1],galaxy.P[frame,i,2],s=5)

    plt.show()



In [7]:
def animate_galaxy(galaxy,r_max,speed=1,orbits_radii = []):    #view_perturber=False,view_perturber_orbit=False

    speed = int(speed)
    while speed>len(galaxy.n_body_system): speed = speed-1

    index_orbits = []
    for r in orbits_radii: 
        index = 0
        r_min = r_max
        for i in range(len(galaxy.n_body_system[0])):
            if np.absolute(r-galaxy.n_body_system[0][i].r)<r_min:
                r_min = np.absolute(r-galaxy.n_body_system[0][i].r)
                index = i
    index_orbits.append(index)

    #create figure and set axes
    fig = plt.figure(figsize=(15, 15)) ; ax = fig.add_subplot(projection='3d')
    ax.set_xlim3d(-r_max, r_max) ; ax.set_ylim3d(-r_max, r_max) ; ax.set_zlim3d(-r_max, r_max)
    ax.set_xlabel('X [parsec]') ; ax.set_ylabel('Y [parsec]') ; ax.set_zlabel('Z [parsec]') ; ax.set_aspect("equal") ;  # Provide starting angle for the view. #ax.view_init(25, 10)
        
    stars = [ ax.scatter(galaxy.P[0][i,0:1], galaxy.P[0][i,1:2], galaxy.P[0][i,2:], color="black", s=0.2) for i in range(galaxy.N-len(galaxy.perturber_index))]   #color=self.colors[i]        
        
    orbits = []
    for j in index_orbits:
        orbits.append(ax.plot(galaxy.P[0,j,0], galaxy.P[0,j,1], galaxy.P[0,j,2])[0])
        
    pert = [] ; orbits_perturber = []
    for k in galaxy.perturber_index:
        pert.append(ax.scatter(galaxy.P[0][k,0:1], galaxy.P[0][k,1:2], galaxy.P[0][k,2:], color="red", s=15))
        orbits_perturber.append(ax.plot(galaxy.P[0,k,0], galaxy.P[0,k,1], galaxy.P[0,k,2],c='red')[0])
                
    def animate(frame): 
        t = galaxy.t[frame*int(speed)]
        plt.title("t = %.2f i.u. (%.2f Myr)" %(t,t*time_conv), fontsize=20) 

        for i in range(galaxy.N-len(galaxy.perturber_index)): 
            stars[i]._offsets3d = (galaxy.P[frame*int(speed)][i,0:1], galaxy.P[frame*int(speed)][i,1:2], galaxy.P[frame*int(speed)][i,2:])
   
        for k in range(len(index_orbits)):
            j = index_orbits[k]
            orbits[k].set_data(galaxy.P[:frame*speed,j,0],galaxy.P[:frame*speed,j,1])
            orbits[k].set_3d_properties(galaxy.P[:frame*speed,j,2])   

        for k in range(len(galaxy.perturber_index)):
            j = galaxy.perturber_index[k]
            pert[k]._offsets3d = (galaxy.P[frame*int(speed)][j,0:1], galaxy.P[frame*int(speed)][j,1:2], galaxy.P[frame*int(speed)][j,2:])
            orbits_perturber[k].set_data(galaxy.P[:frame*speed,j,0],galaxy.P[:frame*speed,j,1])
            orbits_perturber[k].set_3d_properties(galaxy.P[:frame*speed,j,2])   

    anim=animation.FuncAnimation(fig, animate, frames=int(len(galaxy.P)/speed)-1, interval=200) 
    display(HTML(anim.to_jshtml()))
    #if filename != None: ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/homogeneus_collapse/%s_collapse_animation.html" %filename, writer="html")
    #ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/homogeneus_collapse/%s_collapse_animation.html" %self.filename, writer="html")
    #ani.save(filename="/home/robertoinfurna/dynamics_of_stellar_systems/treecode/%s_collapse_animation.gif" %self.filename, writer="pillow")
    plt.close()
  

In [ ]:
def sky_projection(galaxy,r_max,dr,time=0,i=0,alpha=0,gamma=0): 

    if time > galaxy.t[-1]: time=galaxy.t[-1] 
    frame = np.digitize(time,galaxy.t)-1

    beta = np.pi/2-i
    LOS = np.array([cos(alpha)*cos(beta),sin(alpha)*cos(beta),-sin(beta)])
    ra_vector = np.array([cos(alpha)*sin(beta)*sin(gamma) + sin(alpha)*cos(gamma),sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma),cos(beta)*sin(gamma)])
    dec_vector = np.array([cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma),sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma),cos(beta)*cos(gamma)])
    
    ra_r = [] ; dec_r = [] ; v_LOS = []
    for star in galaxy.n_body_system[frame]: 
        if star.m == galaxy.m: #exclude black hole to surface brightness
            ra_r.append(np.dot(star.position,ra_vector))
            dec_r.append(np.dot(star.position,dec_vector))
            v_LOS.append(np.dot(star.velocity,LOS))  

    bins = [np.arange(-r_max, r_max+dr, dr),np.arange(-r_max, r_max+dr, dr)] #+ dr
    LOS_v_matrix = [[ [] for _ in range(len(bins[0]))] for _ in range(len(bins[1]))]
        
    for k in range(galaxy.N-len(galaxy.perturber_index)):
        i,j = np.digitize(ra_r[k],bins[0]), np.digitize(dec_r[k],bins[1]) 
        if i != 0 and j != 0 and i != len(bins[0]) and j != len(bins[1]): 
            LOS_v_matrix[i-1][j-1].append(float(v_LOS[k]))
      
    LOS_v_mean = np.zeros((len(bins[1])-1,len(bins[0])-1)) ; LOS_v_dispersion = np.zeros((len(bins[1])-1,len(bins[0])-1)) ; LOS_v_disp_errors = np.zeros((len(bins[1])-1,len(bins[0])-1))
    for i in range(len(bins[1])-1):
            for j in range(len(bins[0])-1):
                LOS_v_mean[i][j] = np.mean(LOS_v_matrix[j][i])  if len(LOS_v_matrix[j][i]) > 0 else 0
                LOS_v_dispersion[i][j] = np.std(LOS_v_matrix[j][i])  if len(LOS_v_matrix[j][i]) > 1 else 0
                
                #error sigma_LOS
                N_bin = len(LOS_v_matrix[j][i]) 
                #dsigma_dv_mean = -(np.sum(LOS_v_matrix[j][i])-N_bin*np.mean(LOS_v_matrix[j][i]))/(N_bin-1)
                #LOS_v_disp_errors[i][j] = np.sqrt(dsigma_dv_mean**2 + 1/4 * LOS_v_dispersion[i][j]**2 * N_bin/(N_bin-1)**2 ) if len(LOS_v_matrix[j][i]) > 0 else 0
                LOS_v_disp_errors[i][j] = LOS_v_dispersion[i][j]/np.sqrt(2*(N_bin-1))
    
    return bins, ra_r, dec_r, LOS_v_mean, LOS_v_dispersion, LOS_v_disp_errors 



In [ ]:
def jeans_spherical(galaxy,r_max,dr,time): 
        
    if time > galaxy.t[-1]: time=galaxy.t[-1] 
    frame = np.digitize(time,galaxy.t)-1

    i = 0 ; alpha = 0 ; beta = 0  ; gamma = 0
    LOS = np.array([cos(alpha)*cos(beta),sin(alpha)*cos(beta),-sin(beta)])
    ra_vector = np.array([cos(alpha)*sin(beta)*sin(gamma) + sin(alpha)*cos(gamma),sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma),cos(beta)*sin(gamma)])
    dec_vector = np.array([cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma),sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma),cos(beta)*cos(gamma)])

    #radial component
        
    r = np.arange(0, r_max+dr, dr) 
    R = np.arange(0, r_max+dr, dr) 

    v_at_r = [ [] for _ in range(len(r))] 
    v_LOS_R = [ [] for _ in range(len(R))] 

    for star in galaxy.n_body_system[frame]:
        if 0 < star.r < r_max+dr:
            j = np.digitize(star.r,r) -1
            theta = np.arccos(star.position[2]/star.r) 
            phi = np.arctan(star.position[1]/star.position[0])
            r_versor = np.array([np.sin(theta)*np.cos(phi),np.sin(theta)*np.sin(phi),np.cos(theta)])
            #v_r = star.velocity[0]*np.sin(theta)*np.cos(phi)+star.velocity[1]*np.sin(theta)*np.sin(phi)+star.velocity[2]*np.cos(theta)
            v_at_r[j].append(float(np.dot(star.velocity,r_versor))) 

            ra = np.dot(star.position,ra_vector)
            dec = np.dot(star.position,dec_vector)
            p = np.sqrt(float(ra)**2+float(dec)**2)
            k = np.digitize(p,r)-1
            v_LOS_R[k].append(float(np.dot(star.velocity,LOS))) 

    r_bin = []
    R_bin = []
    sigma_r = []  
    error_sigma_r = []
    sigma_LOS = []
    error_sigma_LOS = []

    for i in range(len(r)-1):

        r_bin.append((r[i]+r[i+1])/2)
        
        sigma_r_value = np.std(v_at_r[i],ddof=1)
        sigma_r.append(sigma_r_value)

        #error sigma_r
        N_bin = len(v_at_r[i]) 
        dsigma_dv_mean = -(np.sum(v_at_r[i])-N_bin*np.mean(v_at_r[i]))/(N_bin-1)         
        delta_sigma = np.sqrt(dsigma_dv_mean**2 + 1/4 * sigma_r_value**2 * N_bin/(N_bin-1)**2 )
        error_sigma_r.append(delta_sigma)

        R_bin.append((R[i]+R[i+1])/2)

        sigma_LOS_value = np.std(v_LOS_R[i],ddof=1)
        sigma_LOS.append(sigma_LOS_value)

        #error sigma_LOS
        N_bin = len(v_LOS_R[i]) 
        #dsigma_dv_mean = -(np.sum(v_LOS_R[i])-N_bin*np.mean(v_LOS_R[i]))/(N_bin-1)
        #delta_sigma = np.sqrt(dsigma_dv_mean**2 + 1/4 * sigma_LOS_value**2 * N_bin/(N_bin-1)**2 )
        error_sigma_LOS.append(sigma_LOS_value/np.sqrt(2*(N_bin-1)))
        
    return np.array(r_bin), np.array(R_bin), np.array(sigma_r), np.array(error_sigma_r), np.array(sigma_LOS), np.array(error_sigma_LOS)
